<a href="https://colab.research.google.com/github/rexbrandy/Neural_Networks/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

# Datasets are images stored in `train/` and `val/` folders
data_dir = '/content/drive/MyDrive/neural_network_data/hymenoptera_data'

# Names of each dataset
sets = ['train', 'val']

image_datasets = { x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                for x in sets}

dataloaders = { x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) 
                for x in sets}

# dictionary that holds length of each data set (train, val)
dataset_sizes = { x: len(image_datasets[x]) for x in sets}

class_names = image_datasets['train'].classes
print(class_names)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    '''
        Basic training loop.

        Params:
        model - NN model, holds layers and forward pass function
        criterion - loss function, handles backprop and calculates loss
        optimizer - optimization algorithm, updates params and optim steps
        scheduler - updates learning rate as our loss changes
        num_epochs - how many train/eval loops we should do
    '''

    # Track how long training took
    since = time.time()

    # We will save the best model state in case we move away from it
    best_model_wts = copy.deepcopy(model.state_dict())

    best_acc = 0.0

    for epoch in range(num_epochs):
        # Begin training loops
        print(f'Epoch: {epoch+1}')

        # The training phase will go first
        # then an evaluation phase
        # then epoch will end
        for phase in sets:
            if phase == 'train':
                model.train()
            elif phase == 'val':
                model.eval()
        
            running_loss = 0.0
            running_corrects = 0.0
            
            # Use dataloader to split dataset into batches
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                # track track gradients if train phase
                with torch.set_grad_enabled(phase == 'train'):           
                    output = model(inputs)
                    _, preds = torch.max(output, 1)
                    loss = criterion(output)

                    # Backward pass and optim
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # If it is the end of the training phase update learning rate
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'Phase: {phase}, Loss: {epoch_loss}, Epoch Loss: {epoch_acc}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                '''
                    If it is the training phase 
                    AND the accuracy from this epoch is better than the currect best

                    Update best accuracy
                    Copy model `state_dict`
                '''
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training completed in {(time_elapsed // 60):.0f}m {(time_elapsed % 60):.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model



            

FileNotFoundError: ignored

# New section